In [3]:
import selenium
from selenium import webdriver
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

# 调取Chrome浏览器

In [89]:
import sqlite3

with sqlite3.connect('TmallInformation.db') as con :
    # 读取数据
    Items = pd.read_sql(sql = 'select * from items;',con=con,index_col='index')['text']

查看数据

In [3]:
Items[0:5]

index
0     进口纯牛奶
1      进口酸奶
2    进口成人奶粉
3    进口含乳饮料
4    进口有机牛奶
Name: text, dtype: object

# 抓取三级类目思路

当我们直接访问天猫的时候，我们可以在首页通过解析JSON拿到一二级分类的信息，之后我们想查看二级分类下的具体推荐品牌就需要我们使用搜索功能。

[![knLZQ0.md.png](https://s2.ax1x.com/2019/01/26/knLZQ0.md.png)](https://imgchr.com/i/knLZQ0)

如图所示，我们在搜索框中输入二级分类名称，单击搜索，就可以看到在搜索栏下方就是我们想要的品牌信息，但可惜这部分信息不是JSON，我们需要在单击一下更多才能看到全部的推荐品牌信息。

于是我们就有了获取三级类目的程序流程：
```
for 三级类目 in 品牌数据集：
    1.访问天猫超市首页
    2.在搜索框中输入 三级类目
    3.单击搜索
    4.抓取推荐的品牌信息
 ```
 我们主要使用selenium+chrome的方式登录。万幸，如果从首页通过搜索访问三级类目不需要登录，所以这部分代码实现起来比较简单，在写整个程序的时候我尝试了很多解决登录问题的方法，但很多都失败了，原因是天猫的反爬机制更新的太快了，写爬虫就像猜谜一样，你把所有方法都试一遍总能爬到数据，但对于个人来讲长进不大。

In [113]:
import time
import random
import os 
import re
import pandas as pd
from selenium import webdriver
import threading

import warnings
warnings.filterwarnings("ignore")

RANGE =2
THEADING =1

def InitChrome():
    options = webdriver.ChromeOptions()

    options.add_argument('lang=zh_CN.UTF-8')
#    options.add_argument('user-agent="Mozilla/5.0 (iPod; U; CPU iPhone OS 2_1 like Mac OS X; ja-jp) AppleWebKit/525.18.1 (KHTML, like Gecko) Version/3.1.1 Mobile/5F137 Safari/525.20"')
    prefs = {
        'profile.default_content_setting_values': {
        'images': 2
        }
    }
    options.add_argument('--headless') 
#    options.add_argument('--proxy-server='+ip)
    options.add_argument('disable-infobars')
    options.add_experimental_option('prefs',prefs)
    browser = webdriver.Chrome(chrome_options=options)
    return browser

def getInformation(item,number):
    print('Th-',number,'start',end = '\t')
    print('items is : ',item)
    infmoration = []
    url = 'https://chaoshi.tmall.com/?targetPage=index'

    for i in item:
        print(i)
        browser = InitChrome()
        browser.get(url)
        # 清空搜索栏
        browser.find_element_by_xpath('//*[@id="mq"]').clear()
        # 传入数据
        browser.find_element_by_xpath('//*[@id="mq"]').send_keys(i)
        time.sleep(random.randint(3,5))
        # 点击搜索
        browser.find_element_by_xpath('//*[@id="mallSearch"]/form/fieldset/div/button').click()
        try:
            # 如果有跟多则点击没有就Pass
            browser.find_element_by_xpath('//*[@id="J_RecommendProp"]/div/a').click()
        except:
            pass
        try:
            temp = browser.find_element_by_xpath('//*[@id="J_RecommendProp"]/div').text
        except:
            temp = 'LOST'
            print(i+ ' LOST!!!!\n'+'.*'*40)
            pass
        infmoration.append(temp)

        time.sleep(random.randint(3,5))
        print(i,end='--完成抓取\n')
        browser.quit()
    infmoration = pd.DataFrame(infmoration,index=item.index,columns=['Brands'])
    infmoration.index.name='brandsIndex'
    result = pd.concat((item,infmoration),axis =1)
    
    SaveSQL(result)

def SaveSQL(data):
    with sqlite3.connect('TmallInformation.db') as con:
        data.to_sql(name='brands',con = con ,if_exists='append')
    print('save success')

def ChromeThread(listRange):
    temp = []
    for i in range(len(listRange)//THEADING):
        print('Wheel',i*THEADING*RANGE,'-',(i+1)*THEADING*RANGE)
        temp = listRange[i*THEADING:(i+1)*THEADING]
        count = 0
        pool = []
        for packgae in temp:
            count+=1
            pool.append(threading.Thread(target = getInformation, args = (packgae, count)))
        for j in pool:
            j.start()
        for j in pool:
            j.join()
        print('Wheel ' ,i ,' OVER')

if __name__ =='__main__':
    con = sqlite3.connect('TmallInformation.db')
    sql = 'select * from brands'
    try:
        data= pd.read_sql(con=con,sql = sql,index_col='brandsIndex')
    except:
        print('表为空')
        data = []
    if len(data)>0:
        ItemsValue = Items[~Items.index.isin( data.index)]
    else:
        ItemsValue = Items
    listRange = []
    if len(ItemsValue)>=3:
        for i in range(len(ItemsValue)//RANGE):
            listRange.append(ItemsValue.iloc[i*RANGE:(i+1)*RANGE])
        ChromeThread(listRange)
    elif len(ItemsValue)>=1:
        getInformation(ItemsValue,0)
    else:
        print('Over')

Wheel 0 - 2
Th- 1 start	items is :  index
129    牛肉筋
130     猪肉
Name: text, dtype: object
牛肉筋
牛肉筋--完成抓取
猪肉
猪肉--完成抓取
save success
Wheel  0  OVER
Wheel 2 - 4
Th- 1 start	items is :  index
131       鸡鸭禽
513    积木拼图系列
Name: text, dtype: object
鸡鸭禽
鸡鸭禽 LOST!!!!
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
鸡鸭禽--完成抓取
积木拼图系列
积木拼图系列 LOST!!!!
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
积木拼图系列--完成抓取
save success
Wheel  1  OVER
Wheel 4 - 6
Th- 1 start	items is :  index
514    早教/音乐/智能玩具
515          奶粉辅食
Name: text, dtype: object
早教/音乐/智能玩具
早教/音乐/智能玩具 LOST!!!!
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
早教/音乐/智能玩具--完成抓取
奶粉辅食
奶粉辅食--完成抓取
save success
Wheel  2  OVER
Wheel 6 - 8
Th- 1 start	items is :  index
516    帮宝适
517     好奇
Name: text, dtype: object
帮宝适
帮宝适--完成抓取
好奇
好奇--完成抓取
save success
Wheel  3  OVER
Wheel 8 - 10
Th- 1 start	items is :  index
518    美素佳儿
519      贝亲
Name: text, dtype: 

In [107]:
len(Items)

522

# Brands Feature Engering

In [114]:
con = sqlite3.connect('TmallInformation.db')
sql = 'select * from brands'
data= pd.read_sql(con=con,sql = sql,index_col='brandsIndex')

In [115]:
data = data.drop('Brands',axis = 1).join(data.Brands.str.split('\n',expand=True).iloc[:,1:].stack().reset_index(level = 1,drop=1).rename('Brands'))

In [117]:
data.to_sql(con = con,name='brandsAfFE')